In [1]:
!apt-get update
!apt-get install -y texlive-latex-recommended texlive-fonts-recommended texlive-latex-extra texlive-fonts-extra texlive-science texlive-xetex dvipng

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [83.8 kB]
Get:6 https://cli.github.com/packages stable/main amd64 Packages [356 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,892 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,677 kB]
Get:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,711 kB]
Ge

In [2]:
!apt-get install -y texlive-science texlive-fonts-extra

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
texlive-fonts-extra is already the newest version (2021.20220204-1).
texlive-science is already the newest version (2021.20220204-1).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.


In [3]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 2.1 MB/s eta 0:00:00


**Single Agent**

In [ ]:
import os
import tarfile
import subprocess
import shutil
from typing import TypedDict, List, Optional
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# --- CONFIG ---
MODEL_NAME = "sonar"

# --- 1. Define the Agent State ---
class AgentState(TypedDict):
    tar_path: str
    work_dir: str
    tex_content: str
    beamer_code: str
    pdf_path: str
    available_images: List[str]
    feedback: Optional[str]
    error_log: Optional[str]
    iterations: int

# --- 2. Define the Nodes ---

def extract_node(state: AgentState):
    path = state['tar_path']
    # Use absolute paths to avoid confusion
    extract_to = os.path.abspath(path.replace(".tar.gz", "_work").replace(".tar", "_work"))

    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)
    os.makedirs(extract_to, exist_ok=True)

    with tarfile.open(path, "r:*") as tar:
        tar.extractall(path=extract_to, filter='fully_trusted')

    main_file = None
    image_files = []

    # INDEPENDENT search for files
    for root, _, fs in os.walk(extract_to):
        for f in fs:
            full_path = os.path.join(root, f)
            # 1. Collect images
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.pdf')):
                image_files.append(f)

            # 2. Find the main TeX file (contains \begin{document})
            if f.endswith(".tex"):
                with open(full_path, 'r', errors='ignore') as tex:
                    if "\\begin{document}" in tex.read():
                        main_file = full_path

    if not main_file:
        # Fallback: if no main file found, just take the largest .tex file
        tex_files = [os.path.join(root, f) for root, _, fs in os.walk(extract_to) for f in fs if f.endswith('.tex')]
        if tex_files:
            main_file = max(tex_files, key=os.path.getsize)
        else:
            raise ValueError("Could not find a .tex file in the archive.")

    with open(main_file, 'r', errors='ignore') as f:
        content = f.read()

    return {
        "work_dir": extract_to,
        "tex_content": content,
        "available_images": image_files,
        "iterations": 0
    }

def draft_slides_node(state: AgentState):
    llm = ChatOpenAI(
        api_key="",
        base_url="https://api.perplexity.ai",
        model=MODEL_NAME
    )

    system_msg = (
        "You are a LaTeX Beamer expert. Create a professional 16:9 presentation. "
        "IMPORTANT: Only use standard packages like graphicx, amsmath, and amssymb. "
        "Do NOT use stmaryrd or custom .sty files unless explicitly asked. "
        "Return ONLY the raw LaTeX code beginning with \\documentclass."
    )

    user_prompt = f"Original Paper Content: {state['tex_content'][:5000]}\n"
    user_prompt += f"Available Image Files: {state['available_images']}\n"

    if state.get('feedback'):
        user_prompt += f"\nRefine the previous version. User wants to improve: {state['feedback']}\nPrevious Code: {state['beamer_code']}"

    if state.get('error_log'):
        user_prompt += f"\n\nFIX THIS ERROR: {state['error_log']}"

    response = llm.invoke([SystemMessage(content=system_msg), HumanMessage(content=user_prompt)])

    clean_code = response.content.replace("```latex", "").replace("```", "").strip()
    return {"beamer_code": clean_code, "iterations": state['iterations'] + 1}

def compile_node(state: AgentState):
    tex_file = os.path.join(state['work_dir'], "presentation.tex")
    with open(tex_file, "w") as f:
        f.write(state['beamer_code'])

    # Capture raw output to avoid UnicodeDecodeError
    result = subprocess.run(
        ["pdflatex", "-interaction=nonstopmode", "presentation.tex"],
        cwd=state['work_dir'],
        capture_output=True
    )

    # Safely decode the logs
    stdout_log = result.stdout.decode('utf-8', errors='replace')

    pdf_out = os.path.join(state['work_dir'], "presentation.pdf")

    if result.returncode != 0:
        return {"error_log": stdout_log[-800:], "pdf_path": ""}

    # Double check if PDF actually exists even if return code was 0
    if os.path.exists(pdf_out):
        return {"pdf_path": pdf_out, "error_log": None}
    else:
        return {"error_log": stdout_log[-800:], "pdf_path": ""}

# --- 3. Build Graph Logic ---

def route_after_compile(state: AgentState):
    if state["error_log"] and state["iterations"] < 3:
        print("!! Compiling failed. Agent is self-correcting...")
        return "draft"
    return END

workflow = StateGraph(AgentState)
workflow.add_node("extract", extract_node)
workflow.add_node("draft", draft_slides_node)
workflow.add_node("compile", compile_node)

workflow.set_entry_point("extract")
workflow.add_edge("extract", "draft")
workflow.add_edge("draft", "compile")
workflow.add_conditional_edges("compile", route_after_compile)

app = workflow.compile()

# --- 4. The Agent Runner with Feedback ---

def run_agent(file_path):
    # Initial run
    current_state = {"tar_path": file_path, "feedback": None, "error_log": None, "iterations": 0}

    while True:
        final_state = app.invoke(current_state)

        pdf_path = final_state.get('pdf_path')
        if pdf_path and os.path.exists(pdf_path):
            print(f"\n✅ PDF Generated successfully at: {pdf_path}")
            print("You can download it manually from the file explorer.")
        else:
            print("❌ Failed to compile after retries.")
            print("Current LaTeX Error:", final_state.get('error_log'))

        # HUMAN-IN-THE-LOOP SECTION
        choice = input("\nAre you happy with this PPT? (yes/no): ").strip().lower()
        if choice == 'yes':
            print("Agent stopped. Success!")
            break
        else:
            print("\nWhat should I improve?")
            print("Options: [style, content, images,g maths, more slides]")
            user_critique = input("Your choice: ")

            # Update state and loop back
            current_state = final_state
            current_state["feedback"] = user_critique
            current_state["iterations"] = 0 # Reset iterations for the new attempt
            print("\n🔄 Agent is refining based on your feedback...")

run_agent('/content/arXiv-2602.04039v1.tar.gz')

!! Compiling failed. Agent is self-correcting...
!! Compiling failed. Agent is self-correcting...
❌ Failed to compile after retries.
Current LaTeX Error: utput is active
 [][][][]  
[5]
Overfull \hbox (119.10965pt too wide) has occurred while \output is active
 [][]  \OT1/cmss/m/n/6 ([])  

Overfull \hbox (79.10783pt too wide) has occurred while \output is active
 [][][][]  
[6]
Overfull \hbox (119.10965pt too wide) has occurred while \output is active
 [][]  \OT1/cmss/m/n/6 ([])  

Overfull \hbox (79.10783pt too wide) has occurred while \output is active
 [][][][]  
[7])
Runaway argument?
{frame> \par \begin {frame}{Vulnerability Spawn Rate} \begin {center}\ETC.
! File ended while scanning use of \frame.
<inserted text> 
                \par 
<*> presentation.tex
                    
! Emergency stop.
<*> presentation.tex
                    
!  ==> Fatal error occurred, no output PDF file produced!
Transcript written on presentation.log.


Are you happy with this PPT? (yes/no): yes
A

**Multi-Agent**

In [ ]:
import os
import tarfile
import subprocess
import shutil
from typing import TypedDict, List, Optional
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# --- CONFIG ---
# Recommended models on OpenRouter:
# "anthropic/claude-3.5-sonnet", "google/gemini-2.0-flash-001", "openai/gpt-4o"
MODEL_NAME = "google/gemini-2.0-flash-001"

# --- 1. Define the Agent State ---
class AgentState(TypedDict):
    tar_path: str
    work_dir: str
    tex_content: str
    presentation_plan: str  # New: Stores the outline from the Planner Agent
    beamer_code: str
    pdf_path: str
    available_images: List[str]
    feedback: Optional[str]
    error_log: Optional[str]
    iterations: int

# --- 2. Initialize the LLM (OpenRouter) ---
def get_llm():
    return ChatOpenAI(
        api_key="",
        base_url="https://openrouter.ai/api/v1",
        model=MODEL_NAME
    )

# --- 3. Define the Nodes ---

def extract_node(state: AgentState):
    """
    Extracts the tar file and finds the main .tex source and images.
    """
    path = state['tar_path']
    extract_to = os.path.abspath(path.replace(".tar.gz", "_work").replace(".tar", "_work"))

    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)
    os.makedirs(extract_to, exist_ok=True)

    try:
        with tarfile.open(path, "r:*") as tar:
            tar.extractall(path=extract_to, filter='fully_trusted')
    except Exception as e:
        print(f"Extraction warning: {e}")

    main_file = None
    image_files = []

    for root, _, fs in os.walk(extract_to):
        for f in fs:
            full_path = os.path.join(root, f)
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.pdf')):
                # Store relative path for LaTeX
                rel_path = os.path.relpath(full_path, extract_to)
                image_files.append(rel_path)

            if f.endswith(".tex"):
                with open(full_path, 'r', errors='ignore') as tex:
                    if "\\begin{document}" in tex.read():
                        main_file = full_path

    if not main_file:
        tex_files = [os.path.join(root, f) for root, _, fs in os.walk(extract_to) for f in fs if f.endswith('.tex')]
        if tex_files:
            main_file = max(tex_files, key=os.path.getsize)
        else:
            raise ValueError("Could not find a .tex file in the archive.")

    with open(main_file, 'r', errors='ignore') as f:
        content = f.read()

    print(f"✅ Extracted. Found {len(image_files)} images.")

    return {
        "work_dir": extract_to,
        "tex_content": content,
        "available_images": image_files,
        "iterations": 0
    }

def planner_agent_node(state: AgentState):
    """
    AGENT 1: The Planner.
    Analyzes the paper and produces a structured outline (Plan).
    Does NOT write LaTeX code.
    """
    print("🧠 Planner Agent is structuring the presentation...")
    llm = get_llm()

    system_msg = (
        "You are a Senior Research Communicator. Your goal is to structure a 10-12 slide presentation "
        "based on the provided raw LaTeX paper content.\n"
        "Output a structured textual plan with:\n"
        "1. Slide Title\n"
        "2. Key Bullet Points (content to include)\n"
        "3. Suggested Visuals (if any valid image filenames are listed below).\n"
        "Do NOT write LaTeX code yet. Just the logical flow."
    )

    user_prompt = f"Available Images: {state['available_images']}\n\nRaw Paper Content (Truncated): {state['tex_content'][:15000]}"

    # If there is feedback, the planner re-evaluates the plan
    if state.get('feedback'):
        user_prompt = f"FEEDBACK FROM USER: {state['feedback']}\n\n" + user_prompt

    response = llm.invoke([SystemMessage(content=system_msg), HumanMessage(content=user_prompt)])

    return {"presentation_plan": response.content}

def developer_agent_node(state: AgentState):
    """
    AGENT 2: The Developer.
    Takes the Plan and writes the valid LaTeX Beamer code.
    """
    print("💻 Developer Agent is writing LaTeX code...")
    llm = get_llm()

    system_msg = (
        "You are a LaTeX Beamer Expert. Convert the provided 'Presentation Plan' into a high-quality, "
        "compilable LaTeX Beamer presentation.\n"
        "RULES:\n"
        "1. Use \\documentclass{beamer}.\n"
        "2. Use \\usetheme{Madrid} or similar clean theme.\n"
        "3. ONLY use standard packages: graphicx, amsmath, amssymb, hyperref.\n"
        "4. Do NOT use custom .sty files or bibliography files (.bib).\n"
        "5. If images are used, ensure the filename matches exactly what is provided in the list.\n"
        "6. Wrap the code in ```latex ... ``` blocks.\n"
        "7. Ensure the code is complete (ends with \\end{document})."
    )

    user_prompt = (
        f"Here is the Presentation Plan:\n{state['presentation_plan']}\n\n"
        f"List of valid image files you can include using \\includegraphics: {state['available_images']}\n"
    )

    if state.get('error_log'):
        user_prompt += f"\n\n⚠️ PREVIOUS COMPILATION ERROR: {state['error_log']}\nFix the code based on this error."

    response = llm.invoke([SystemMessage(content=system_msg), HumanMessage(content=user_prompt)])

    # Clean formatting to get raw code
    content = response.content
    if "```latex" in content:
        code = content.split("```latex")[1].split("```")[0].strip()
    elif "```" in content:
        code = content.split("```")[1].split("```")[0].strip()
    else:
        code = content.strip()

    return {"beamer_code": code, "iterations": state['iterations'] + 1}

def compile_node(state: AgentState):
    """
    Compiles the LaTeX code using pdflatex.
    """
    print("⚙️ Compiling PDF...")
    tex_file = os.path.join(state['work_dir'], "presentation.tex")

    # Write the code
    with open(tex_file, "w") as f:
        f.write(state['beamer_code'])

    # Compile (run twice for TOC/Labels if needed, but once is usually enough for draft)
    result = subprocess.run(
        ["pdflatex", "-interaction=nonstopmode", "presentation.tex"],
        cwd=state['work_dir'],
        capture_output=True
    )

    stdout_log = result.stdout.decode('utf-8', errors='replace')
    pdf_out = os.path.join(state['work_dir'], "presentation.pdf")

    # Check logic
    if os.path.exists(pdf_out) and result.returncode == 0:
        return {"pdf_path": pdf_out, "error_log": None}
    elif os.path.exists(pdf_out):
        # PDF exists but might have minor errors
        return {"pdf_path": pdf_out, "error_log": stdout_log[-1000:]}
    else:
        # Critical failure
        return {"error_log": stdout_log[-1000:], "pdf_path": ""}

# --- 4. Build Graph Logic ---

def route_after_compile(state: AgentState):
    if state["pdf_path"] == "" and state["iterations"] < 3:
        print(f"❌ Compilation failed. Retrying (Attempt {state['iterations']}/3)...")
        # If compilation fails, go back to Developer to fix syntax
        return "developer"
    return END

workflow = StateGraph(AgentState)

# Add Nodes
workflow.add_node("extract", extract_node)
workflow.add_node("planner", planner_agent_node)
workflow.add_node("developer", developer_agent_node)
workflow.add_node("compile", compile_node)

# Add Edges
workflow.set_entry_point("extract")
workflow.add_edge("extract", "planner")
workflow.add_edge("planner", "developer")
workflow.add_edge("developer", "compile")

# Conditional Edge: If compile fails, loop back to developer. Else finish.
workflow.add_conditional_edges("compile", route_after_compile, {"developer": "developer", END: END})

app = workflow.compile()

# --- 5. The Runner ---

def run_multi_agent_system(file_path):
    print(f"🚀 Starting Multi-Agent PPT Generator for {os.path.basename(file_path)}...")

    current_state = {
        "tar_path": file_path,
        "feedback": None,
        "error_log": None,
        "iterations": 0
    }

    while True:
        # Run the graph
        final_state = app.invoke(current_state)

        pdf_path = final_state.get('pdf_path')
        if pdf_path and os.path.exists(pdf_path):
            print(f"\n✅ SUCCESS! PDF generated at: {pdf_path}")
        else:
            print("\n❌ FAILED to generate a valid PDF after retries.")
            if final_state.get('error_log'):
                print("Last LaTeX Error snippet:\n", final_state['error_log'])

        # Human-in-the-Loop Feedback
        choice = input("\nAre you happy with this PPT? (yes/no): ").strip().lower()
        if choice == 'yes':
            print("🎉 Process Complete.")
            break
        else:
            user_critique = input("Enter your feedback (e.g., 'Make it more detailed', 'Fix slide 3'): ")
            print("\n🔄 Restarting Agents with your feedback...")

            # Reset state for refinement
            # We go back to the Planner if content changes are needed
            current_state = final_state
            current_state["feedback"] = user_critique
            current_state["iterations"] = 0
            current_state["error_log"] = None # Clear old errors so we don't confuse the model

            # Re-run starting from Planner to incorporate content feedback
            # (Note: In a more complex graph, you might route feedback specifically to Planner OR Developer)
            # For simplicity, we restart the generation flow with the new context.
            # We must manually re-invoke the app logic, effectively looping this 'while' block.

# Upload a tar.gz file to Colab first!
run_multi_agent_system('/content/arXiv-2602.04039v1.tar.gz')

🚀 Starting Multi-Agent PPT Generator for arXiv-2602.04039v1.tar.gz...
✅ Extracted. Found 6 images.
🧠 Planner Agent is structuring the presentation...
💻 Developer Agent is writing LaTeX code...
⚙️ Compiling PDF...

✅ SUCCESS! PDF generated at: /content/arXiv-2602.04039v1_work/presentation.pdf

Are you happy with this PPT? (yes/no): no
Enter your feedback (e.g., 'Make it more detailed', 'Fix slide 3'): make it more attractive, its very simple currently

🔄 Restarting Agents with your feedback...
✅ Extracted. Found 6 images.
🧠 Planner Agent is structuring the presentation...
💻 Developer Agent is writing LaTeX code...
⚙️ Compiling PDF...

✅ SUCCESS! PDF generated at: /content/arXiv-2602.04039v1_work/presentation.pdf

Are you happy with this PPT? (yes/no): yes
🎉 Process Complete.


**With frontend**

In [11]:
!pip install gradio langgraph langchain-openai
# For pdflatex (if not installed, e.g., in Colab):
!apt update && apt install -y texlive texlive-latex-extra texlive-fonts-recommended

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Fetched 384 kB in 1s (262 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
53 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspe

In [ ]:
import os
import tarfile
import subprocess
import shutil
from typing import TypedDict, List, Optional
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
import gradio as gr

# --- CONFIG ---
# Replace with your actual OpenRouter Key
MODEL_NAME = "google/gemini-2.0-flash-001"

# --- 1. Define the Agent State ---
class AgentState(TypedDict):
    tar_path: str
    work_dir: str
    tex_content: str
    presentation_plan: str
    beamer_code: str
    pdf_path: str
    available_images: List[str]
    feedback: Optional[str]
    error_log: Optional[str]
    iterations: int

# --- 2. Initialize the LLM ---
def get_llm():
    return ChatOpenAI(
        api_key="",
        base_url="https://openrouter.ai/api/v1",
        model=MODEL_NAME
    )

# --- 3. Define the Nodes ---

def extract_node(state: AgentState):
    path = state['tar_path']
    extract_to = os.path.abspath(path.replace(".tar.gz", "_work").replace(".tar", "_work"))

    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)
    os.makedirs(extract_to, exist_ok=True)

    try:
        with tarfile.open(path, "r:*") as tar:
            tar.extractall(path=extract_to, filter='fully_trusted')
    except Exception as e:
        print(f"Extraction warning: {e}")

    main_file = None
    image_files = []

    for root, _, fs in os.walk(extract_to):
        for f in fs:
            full_path = os.path.join(root, f)
            if f.lower().endswith(('.png', '.jpg', '.jpeg', '.pdf')):
                rel_path = os.path.relpath(full_path, extract_to)
                image_files.append(rel_path)

            if f.endswith(".tex"):
                with open(full_path, 'r', errors='ignore') as tex:
                    if "\\begin{document}" in tex.read():
                        main_file = full_path

    if not main_file:
        tex_files = [os.path.join(root, f) for root, _, fs in os.walk(extract_to) for f in fs if f.endswith('.tex')]
        if tex_files:
            main_file = max(tex_files, key=os.path.getsize)
        else:
            raise ValueError("Could not find a .tex file in the archive.")

    with open(main_file, 'r', errors='ignore') as f:
        content = f.read()

    print(f"✅ Extracted. Found {len(image_files)} images.")

    return {
        "work_dir": extract_to,
        "tex_content": content,
        "available_images": image_files,
        "iterations": 0
    }

def planner_agent_node(state: AgentState):
    print("🧠 Planner Agent is structuring the presentation...")
    llm = get_llm()

    system_msg = (
        "You are a Senior Research Communicator. Your goal is to structure a 10-12 slide presentation "
        "based on the provided raw LaTeX paper content.\n"
        "Output a structured textual plan with:\n"
        "1. Slide Title\n"
        "2. Key Bullet Points (content to include)\n"
        "3. Suggested Visuals (if any valid image filenames are listed below).\n"
        "Do NOT write LaTeX code yet. Just the logical flow."
    )

    user_prompt = f"Available Images: {state['available_images']}\n\nRaw Paper Content (Truncated): {state['tex_content'][:15000]}"

    if state.get('feedback'):
        user_prompt = f"FEEDBACK FROM USER: {state['feedback']}\n\n" + user_prompt

    response = llm.invoke([SystemMessage(content=system_msg), HumanMessage(content=user_prompt)])

    return {"presentation_plan": response.content}

def developer_agent_node(state: AgentState):
    print("💻 Developer Agent is writing LaTeX code...")
    llm = get_llm()

    system_msg = (
        "You are a LaTeX Beamer Expert. Convert the provided 'Presentation Plan' into a high-quality, "
        "compilable LaTeX Beamer presentation.\n"
        "RULES:\n"
        "1. Use \\documentclass{beamer}.\n"
        "2. Use \\usetheme{Madrid} or similar clean theme.\n"
        "3. ONLY use standard packages: graphicx, amsmath, amssymb, hyperref.\n"
        "4. Do NOT use custom .sty files or bibliography files (.bib).\n"
        "5. If images are used, ensure the filename matches exactly what is provided in the list.\n"
        "6. Wrap the code in ```latex ... ``` blocks.\n"
        "7. Ensure the code is complete (ends with \\end{document})."
    )

    user_prompt = (
        f"Here is the Presentation Plan:\n{state['presentation_plan']}\n\n"
        f"List of valid image files you can include using \\includegraphics: {state['available_images']}\n"
    )

    if state.get('error_log'):
        user_prompt += f"\n\n⚠️ PREVIOUS COMPILATION ERROR: {state['error_log']}\nFix the code based on this error."

    response = llm.invoke([SystemMessage(content=system_msg), HumanMessage(content=user_prompt)])

    content = response.content
    if "```latex" in content:
        code = content.split("```latex")[1].split("```")[0].strip()
    elif "```" in content:
        code = content.split("```")[1].split("```")[0].strip()
    else:
        code = content.strip()

    return {"beamer_code": code, "iterations": state['iterations'] + 1}

def compile_node(state: AgentState):
    print("⚙️ Compiling PDF...")
    tex_file = os.path.join(state['work_dir'], "presentation.tex")

    with open(tex_file, "w") as f:
        f.write(state['beamer_code'])

    result = subprocess.run(
        ["pdflatex", "-interaction=nonstopmode", "presentation.tex"],
        cwd=state['work_dir'],
        capture_output=True
    )

    stdout_log = result.stdout.decode('utf-8', errors='replace')
    pdf_out = os.path.join(state['work_dir'], "presentation.pdf")

    if os.path.exists(pdf_out) and result.returncode == 0:
        return {"pdf_path": pdf_out, "error_log": None}
    elif os.path.exists(pdf_out):
        return {"pdf_path": pdf_out, "error_log": stdout_log[-1000:]}
    else:
        return {"error_log": stdout_log[-1000:], "pdf_path": ""}

# --- 4. Build Graph Logic ---

def route_after_compile(state: AgentState):
    if state["pdf_path"] == "" and state["iterations"] < 3:
        print(f"❌ Compilation failed. Retrying (Attempt {state['iterations']}/3)...")
        return "developer"
    return END

workflow = StateGraph(AgentState)
workflow.add_node("extract", extract_node)
workflow.add_node("planner", planner_agent_node)
workflow.add_node("developer", developer_agent_node)
workflow.add_node("compile", compile_node)
workflow.set_entry_point("extract")
workflow.add_edge("extract", "planner")
workflow.add_edge("planner", "developer")
workflow.add_edge("developer", "compile")
workflow.add_conditional_edges("compile", route_after_compile, {"developer": "developer", END: END})
app = workflow.compile()

# --- Gradio Frontend ---

def process_tar(uploaded_file, feedback_text, current_state):
    if uploaded_file is None and current_state is None:
        return None, "Please upload a compressed archive file.", None

    if uploaded_file is not None:
        file_path = uploaded_file.name  # Gradio uploads to temp file; use .name for path
        # Validate if it's a tar file (compressed or not)
        if not tarfile.is_tarfile(file_path):
            return None, "Invalid file: Not a valid tar archive (supports .tar, .tar.gz, etc.).", None
        current_state = {
            "tar_path": file_path,
            "feedback": feedback_text if feedback_text else None,
            "error_log": None,
            "iterations": 0
        }
    else:
        # Feedback on existing state
        current_state["feedback"] = feedback_text if feedback_text else None
        current_state["iterations"] = 0
        current_state["error_log"] = None

    try:
        final_state = app.invoke(current_state)
        pdf_path = final_state.get('pdf_path')
        if pdf_path and os.path.exists(pdf_path):
            status = f"✅ PDF generated! Download below.\nPlan:\n{final_state['presentation_plan'][:500]}...\nCode snippet:\n{final_state['beamer_code'][:500]}..."
            return pdf_path, status, final_state  # Return PDF path for download, status, and updated state
        else:
            error = final_state.get('error_log', 'Unknown error')
            return None, f"❌ Failed: {error}", current_state
    except Exception as e:
        return None, f"Error: {str(e)}", current_state

with gr.Blocks(title="LaTeX to Beamer Presentation Generator") as demo:
    gr.Markdown("# Upload Compressed Archive File to Generate Presentation PDF")

    state = gr.State(None)  # Persist agent state across interactions

    uploaded_file = gr.File(label="Upload Compressed Archive File")  # Removed file_types to allow any, validate in code
    feedback = gr.Textbox(label="Feedback (e.g., 'Make slides more detailed')", placeholder="Optional for refinements")
    process_btn = gr.Button("Generate PDF")

    pdf_output = gr.File(label="Download Generated PDF")
    status = gr.Textbox(label="Status", interactive=False)

    process_btn.click(
        process_tar,
        inputs=[uploaded_file, feedback, state],
        outputs=[pdf_output, status, state]
    )

demo.launch(share=True)  # share=True for public link in Colab

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://577b5ce9edb2bf5ee3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
